In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import re
from pandas.io.json import json_normalize
from pandas.api.types import is_string_dtype
import os
import json

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

In [3]:
PATH = '../data/'

## todo:

1. <s> Figure out how to get stuff out of the JSON columns </s>
3. <s> Prepare the dependent variable so it predicts what it needs to </s>
4. <s> Make sure categorical variables are correctly encoded </s>
5. <s> one-hot encode variables with cardinality <7 </s>
6. <s> Get variables out of the date variable </s>
7. Not sure I understand the distribution of "totals.transactionRevenue". Why is the mode around 16? 
7. <s> Deal with missing values </s>
8. Preprocess the test data, too. Run the transformations on the joint df and then divide



# Load the data

In [4]:
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

In [5]:
# Get data hidden inside json columns out into separate columns 
# credit: julian3833 https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields

def load_df(csv_path=f'{PATH}train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str',
                            'totals.transactionRevenu': 'int64'
                           },
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [6]:
nrows=None
# nrows=2000

In [7]:
%time df = load_df(nrows=nrows)

Loaded train.csv. Shape: (903653, 55)
CPU times: user 2min 35s, sys: 13 s, total: 2min 48s
Wall time: 2min 46s


In [8]:
df.describe()

,date,visitId,visitNumber,visitStartTime
count,9.036530e+05,9.036530e+05,903653.000000,9.036530e+05
mean,2.016589e+07,1.485007e+09,2.264897,1.485007e+09
std,4.697698e+03,9.022124e+06,9.283735,9.022124e+06
min,2.016080e+07,1.470035e+09,1.000000,1.470035e+09
25%,2.016103e+07,1.477561e+09,1.000000,1.477561e+09
50%,2.017011e+07,1.483949e+09,1.000000,1.483949e+09
75%,2.017042e+07,1.492759e+09,1.000000,1.492759e+09
max,2.017080e+07,1.501657e+09,395.000000,1.501657e+09


In [9]:
#date span
df.date.min(), df.date.max()

(20160801, 20170801)

# Add date variables

In [10]:
# extract date vars. Adapted from the fastai library

def add_datepart(df, fldname, drop=False, time=False):
    
    fld = df[fldname]
    fld_dtype = fld.dtype

    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, format='%Y%m%d')
        
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: 
        attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: 
        df[targ_pre + n] = getattr(fld.dt, n.lower())
        
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    
    if drop: 
        df.drop(fldname, axis=1, inplace=True)

In [11]:
add_datepart(df, 'date')

# Converting string variables into category variables.

In [12]:
cat_vars = []
for n,c in df.items():
    if is_string_dtype(c): 
        cat_vars.append(n)
        df[n] = c.astype('category').cat.as_ordered()

# A quick look at the missing values

In [13]:
df.isnull().sum().sort_index()/len(df)

Day                                                  0.000000
Dayofweek                                            0.000000
Dayofyear                                            0.000000
Elapsed                                              0.000000
Is_month_end                                         0.000000
Is_month_start                                       0.000000
Is_quarter_end                                       0.000000
Is_quarter_start                                     0.000000
Is_year_end                                          0.000000
Is_year_start                                        0.000000
Month                                                0.000000
Week                                                 0.000000
Year                                                 0.000000
channelGrouping                                      0.000000
date                                                 0.000000
device.browser                                       0.000000
device.b

In [14]:
df = df.replace([np.inf, -np.inf], np.nan)

In [15]:
df['totals.transactionRevenue'].dtype

CategoricalDtype(categories=['10000', '100000000', '100050000', '1000780000',
                  '1001590000', '100220000', '100260000', '100350000',
                  '100440000', '1004490000',
                  ...
                  '99850000', '998750000', '9990000', '99900000', '99950000',
                  '999500000', '99960000', '99970000', '99980000', '99990000'],
                 ordered=True)

In [16]:
df['totals.transactionRevenue'].value_counts(dropna=False)


NaN          892138
16990000        256
18990000        189
33590000        187
44790000        170
13590000        135
55990000        122
19990000        116
15990000         98
15190000         93
19190000         92
10990000         84
59990000         81
24990000         77
79990000         65
27190000         64
27180000         62
33980000         54
39990000         51
67180000         46
1990000          44
28780000         40
21990000         40
35980000         39
30390000         39
17590000         38
31990000         37
27980000         37
30380000         37
12990000         37
              ...  
437070000         1
43710000          1
43720000          1
43780000          1
438020000         1
43080000          1
43850000          1
438540000         1
4390000           1
439000000         1
43930000          1
43950000          1
43640000          1
436300000         1
43610000          1
436020000         1
435910000         1
43590000          1
435860000         1


In [17]:
# df.info(null_counts=True)
cols_with_nulls = []
for col in df.columns: 
    if df[col].hasnans: 
        cols_with_nulls.append(col)
        
cols_with_nulls

['totals.bounces',
 'totals.newVisits',
 'totals.pageviews',
 'totals.transactionRevenue',
 'trafficSource.adContent',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'trafficSource.adwordsClickInfo.gclId',
 'trafficSource.adwordsClickInfo.isVideoAd',
 'trafficSource.adwordsClickInfo.page',
 'trafficSource.adwordsClickInfo.slot',
 'trafficSource.campaignCode',
 'trafficSource.isTrueDirect',
 'trafficSource.keyword',
 'trafficSource.referralPath']

In [18]:
df['totals.bounces'].value_counts(dropna=False)

NaN    453023
1      450630
Name: totals.bounces, dtype: int64

In [19]:
for col in cols_with_nulls:
    if pd.api.types.is_numeric_dtype(df[col]):
        df[col+'_na'] = pd.isnull(df[col])
        df[col] = df[col].fillna(df[col].median())
    elif pd.api.types.is_categorical_dtype(df[col]):
        df[col+'_na'] = pd.isnull(df[col])
        df[col] =  df[col].cat.add_categories(["-1"])
        df[col] = df[col].fillna("-1")




In [20]:
cols_with_nulls = []
for col in df.columns: 
    if df[col].hasnans: 
        cols_with_nulls.append(col)
        
cols_with_nulls

[]

In [21]:
df['totals.bounces'].value_counts(dropna=False)

-1    453023
1     450630
Name: totals.bounces, dtype: int64

In [22]:
df['totals.bounces'].value_counts(dropna=False)

-1    453023
1     450630
Name: totals.bounces, dtype: int64

# One-hot encoding categorical variables with cardinality <= 15.

In [23]:
def dummify(df, min_cardinality=0, max_cardinality=15):

    to_dummify = []
    
    for n,c in df.items():
        
        if str(df.dtypes[n]) == 'category':
            if ((len(df[n].cat.categories)) > min_cardinality) \
            & ((len(df[n].cat.categories)) <= max_cardinality):
                to_dummify.append(n)
                
    dummified = pd.get_dummies(df[to_dummify], dummy_na=True)
    dummified = pd.concat([dummified, df], axis=1)
    dummified_df = dummified.drop(to_dummify, axis=1)
    
    return to_dummify, dummified_df

In [24]:
dumd_vars, df = dummify(df)

In [25]:
#These are the remaining, not dummified categorical variables.
#Let's convert them to ints (otherwise sklearn's random forest is grumpy)

remaining_cat_vars = (list(set(cat_vars) - set(dumd_vars))) 

for var in remaining_cat_vars:
    df[var] = df[var].cat.codes

# Prepare the dependent variable (log-transform)

In [26]:
# Prepare the dependent variable (log)

df['totals.transactionRevenue'] = pd.to_numeric(df['totals.transactionRevenue'], errors='coerce')
df['totals.transactionRevenue'] = df['totals.transactionRevenue'].fillna(0)
df["totals.transactionRevenue"] = np.log1p(df["totals.transactionRevenue"])

In [27]:
df['totals.transactionRevenue'].value_counts(dropna=False)

8.581669    892138
7.123673       256
7.285507       189
7.903227       187
8.118207       170
6.598509       135
8.263591       122
7.368970       116
6.999423        98
6.896694        93
7.302496        92
5.525453        84
8.306719        81
7.633853        77
8.473032        65
7.719574        64
7.717796        62
7.913887        54
8.040125        51
8.370085        46
7.356918        44
7.492203        40
7.771067        40
7.822044        39
7.958577        39
7.171657        38
6.487684        37
7.821242        37
7.743703        37
7.864036        37
             ...  
6.900731         1
6.901737         1
6.902743         1
6.904751         1
6.890609         1
6.905753         1
6.906755         1
6.907755         1
6.908755         1
6.909753         1
6.910751         1
6.891626         1
6.889591         1
6.873164         1
6.880384         1
6.874198         1
6.875232         1
6.876265         1
6.877296         1
6.878326         1
6.879356         1
6.881411    

# Save the preprocessed dataset

In [28]:
df.to_hdf(f'{PATH}'+"preprocessed_df.hdf", key="df")

# _tmp